In [5]:
import pynwb
import os
import numpy as np

# DataJoint and DataJoint schema
import datajoint as dj

dj.config["filepath_checksum_size_limit"] = 1 * 1024**2

## We also import a bunch of tables so that we can call them easily
from spyglass.common import (
    ElectrodeGroup,
    Electrode,
    FirFilterParameters,
    IntervalList,
    Session,
    Task,
    TaskEpoch,
    Nwbfile,
    AnalysisNwbfile,
    NwbfileKachery,
    AnalysisNwbfileKachery,
    interval_list_contains,
    interval_list_contains_ind,
    interval_list_excludes,
    interval_list_excludes_ind,
    interval_list_intersect,
    get_electrode_indices,
    Raw
)

from spyglass.lfp.v1 import (
    LFPElectrodeGroup,
    LFPSelection,
    LFPV1,
)
from spyglass.lfp.analysis.v1 import LFPBandSelection,LFPBandV1
from spyglass.lfp.lfp_merge import LFPOutput
import os
import warnings
from tqdm import tqdm
warnings.simplefilter("ignore", category=DeprecationWarning)
warnings.simplefilter("ignore", category=ResourceWarning)

In [6]:
FirFilterParameters()#.fetch('filter_name')

filter_name descriptive name of this filter,filter_sampling_rate sampling rate for this filter,filter_type,filter_low_stop lowest frequency for stop band for low frequency side of filter,filter_low_pass lowest frequency for pass band of low frequency side of filter,filter_high_pass highest frequency for pass band for high frequency side of filter,filter_high_stop highest frequency for stop band of high frequency side of filter,filter_comments comments about the filter,filter_band_edges numpy array containing the filter bands (redundant with individual parameters),filter_coeff numpy array containing the filter coefficients
Delta 0.5-4 Hz,1000,lowpass,0.25,0.5,4.0,5.0,delta filter for 1 KHz data,=BLOB=,=BLOB=
"Delta 0.5-4 Hz pass, 0.25-4.5 Hz stop",1000,lowpass,0.25,0.5,4.0,4.5,revised delta filter for 1 KHz data,=BLOB=,=BLOB=
Fast Gamma 65-100 Hz,1000,bandpass,55.0,65.0,100.0,110.0,slow gamma filter for 1 Khz data,=BLOB=,=BLOB=
LFP 0-400 Hz,1000,lowpass,0.0,0.0,400.0,425.0,LFP filter for referencing,=BLOB=,=BLOB=
LFP 0-400 Hz,20000,lowpass,0.0,0.0,400.0,425.0,standard LFP filter for 20 KHz data,=BLOB=,=BLOB=
LFP 0-400 Hz,30000,lowpass,0.0,0.0,400.0,425.0,standard LFP filter for 20 KHz data,=BLOB=,=BLOB=
LFP 300-495 Hz,1000,bandpass,275.0,300.0,495.0,499.9,estimated EMG LFP filter for 1KHz data,=BLOB=,=BLOB=
Ripple 150-250 Hz,1000,lowpass,140.0,150.0,250.0,260.0,ripple band filter for 1 kHz data,=BLOB=,=BLOB=
ripple_150_250,1000,bandpass,140.0,150.0,250.0,260.0,"150-250 Hz pass, 140-260 Hz stop",=BLOB=,=BLOB=
Slow Gamma 25-55 Hz,1000,bandpass,20.0,25.0,55.0,60.0,slow gamma filter for 1 Khz data,=BLOB=,=BLOB=


## Define Animls


In [9]:
# probe_animals=[
#     #Banner
#     "Banner20211221_.nwb",
#     "Banner20211222_.nwb",
#     "Banner20211227_.nwb",
#     "Banner20211228_.nwb",
#     "Banner20211230_.nwb",
#     "Banner20211231_.nwb",
#     "Banner20220103_.nwb",
#     # "Banner20220105_.nwb", #Raw Sampling rate = 29995.0 Hz??
#     "Banner20220123_.nwb",
#     "Banner20220124_.nwb",
#     "Banner20220126_.nwb",
#     "Banner20220215_.nwb",
#     "Banner20220216_.nwb",
#     "Banner20220223_.nwb",
#     "Banner20220224_.nwb",
#     "Banner20220225_.nwb",
#     #Danvers
#     # "Danvers20211109_.nwb", #Danvers not imported to spyglass
#     # "Danvers20211110_.nwb",
#     # "Danvers20211111_.nwb",
#     # "Danvers20211112_.nwb",
#     # "Danvers20211113_.nwb",
#     # "Danvers20211114_.nwb",
#     #Odins
#     "Odins20220107_.nwb",
#     "Odins20220110_.nwb",
#     "Odins20220111_.nwb",
#     "Odins20220113_.nwb",
#     "Odins20220118_.nwb",
#     "Odins20220119_.nwb",
#     "Odins20220123_.nwb",
#     "Odins20220124_.nwb",
#     "Odins20220126_.nwb",
#     "Odins20220216_.nwb",
# ]


probe_animal_names = ["Banner","Danvers","Odins"]
from spyglass.common import Session
probe_animals = []
[probe_animals.extend((Session() & {"subject_id":animal}).fetch("nwb_file_name")) for animal in probe_animal_names]
probe_animals.pop(probe_animals.index('samtest20230817_.nwb'))
probe_animals.pop(probe_animals.index('samtestb20230817_.nwb'))

#write permission?
analysis_path = "/stelmo/nwb/analysis/"
for nwb_file in probe_animals:
    if not os.path.isdir(analysis_path+nwb_file.split('_')[0]):
        print(f'No analysis yet for: {nwb_file}')
    elif not os.access(analysis_path+nwb_file.split('_')[0],os.W_OK):
        print(f'No write permissions for: {nwb_file}nwb_file')



# for nwb_file in probe_animals:
#     print(LFP() & {'nwb_file_name':nwb_file})

# Run Probe Animals LFP

In [6]:
nwb_file_name_list = probe_animals#["Banner20220224_.nwb"]
filter_name = "LFP 0-400 Hz"
filter_sampling_rate = 30000
# band_filter_name = "Theta 5-11 Hz"
# band_sampling_rate = 100
# nwb_file_name_list = ["SB2spikegadgets20220224_.nwb"]

analysis_path = "/stelmo/nwb/analysis/"
skipped_sessions = []
for nwb_file_name in nwb_file_name_list:
    #skip session if you don't have write privledges
    if os.path.isdir(analysis_path+nwb_file_name.split('_')[0]) and (not os.access(analysis_path+nwb_file_name.split('_')[0],os.W_OK)):
        skipped_sessions.append(nwb_file_name)
        continue
    print(f'Beginning: {nwb_file_name}')
    #want to run on all electrodes:
    electrode_ids = (Electrode & {"nwb_file_name": nwb_file_name}).fetch(
    "electrode_id")
    lfp_electrode_group_name = "full_probe"
    lfp_eg_key = {
        "nwb_file_name": nwb_file_name,
        "lfp_electrode_group_name": lfp_electrode_group_name,
        }
    LFPElectrodeGroup.create_lfp_electrode_group(
        nwb_file_name=nwb_file_name,
        group_name=lfp_electrode_group_name,
        electrode_list=electrode_ids,  
        )
    #Loop through all position intervals
    for interval_list_name in tqdm((IntervalList& {"nwb_file_name": nwb_file_name,}).fetch('interval_list_name')):        
        #make selection entry
        if ('pos' not in interval_list_name) or ('lfp' in interval_list_name) or (not (interval_list_name[:3]=='pos')): 
            continue
        print(interval_list_name)
        
        
        lfp_s_key = lfp_eg_key.copy()
        lfp_s_key["target_interval_list_name"] = interval_list_name
        lfp_s_key["filter_name"] = filter_name
        lfp_s_key["filter_sampling_rate"] = (Raw() & {"nwb_file_name":nwb_file_name}).fetch1("sampling_rate")#filter_sampling_rate
        LFPSelection.insert1(lfp_s_key, skip_duplicates=True)
        LFPSelection()& {'nwb_file_name': nwb_file_name}
        #populate the lfp
        LFPV1().populate(lfp_s_key)

        # #Make band Filters
        # # set the reference to -1 to indicate no reference for all channels
        # ref_elect = [-1]
        # # we also need the uuid for the LFP object
        # lfp_id = (LFPOutput.LFP & lfp_s_key).fetch1("lfp_id")

        # LFPBandSelection().set_lfp_band_electrodes(
        #     nwb_file_name=nwb_file_name,
        #     lfp_id=lfp_id,
        #     electrode_list=electrode_ids,
        #     filter_name=band_filter_name,
        #     interval_list_name=interval_list_name,
        #     reference_electrode_list=ref_elect,
        #     lfp_band_sampling_rate=band_sampling_rate,
        # )
        # lfp_b_key = (
        #     LFPBandSelection & {"lfp_id": lfp_id, "filter_name": band_filter_name , 'target_interval_list_name':interval_list_name}
        # ).fetch1("KEY")
        # LFPBand().populate(LFPBandSelection() & {"nwb_file_name": nwb_file_name})
        # break



Beginning: SB2spikegadgets20220224_.nwb


  0%|                                                     | 0/5 [00:00<?, ?it/s]

pos 0 valid times


[2023-09-13 11:48:16,616][WARNING]: Skipped checksum for file with hash: 320bd42c-6d75-7265-61dc-ff03cd6339c4, and path: /stelmo/nwb/raw/SB2spikegadgets20220224_.nwb


LFP: found 2 intervals > 1.0 sec long.
Writing new NWB file SB2spikegadgets20220224_QO0PXC2P5S.nwb


Filtering data
Interval 0: loading data into memory
Interval 1: loading data into memory


 60%|███████████████████████████                  | 3/5 [01:10<00:46, 23.43s/it][2023-09-13 11:49:26,613][WARNING]: Skipped checksum for file with hash: 320bd42c-6d75-7265-61dc-ff03cd6339c4, and path: /stelmo/nwb/raw/SB2spikegadgets20220224_.nwb


pos 1 valid times
LFP: found 2 intervals > 1.0 sec long.
Writing new NWB file SB2spikegadgets20220224_R2MD24U7B7.nwb
Filtering data
Interval 0: loading data into memory
Interval 1: loading data into memory


100%|█████████████████████████████████████████████| 5/5 [01:49<00:00, 21.86s/it]


In [28]:
# #Make a new filter for Banner20220105, which has a sampling rate of 29995 instead of 30000. One time operation
# {"filter_name":filter_name, "sampling_rate":29995,"filter_type":"lowpass",
#  "filter_low_stop":0, "filter_low_pass":0, "filter_high_pass":400, "filter_high_stop":425,
#  "filter_comments": "adapted LFP filter for Banner20220105"}

# FirFilterParameters().add_filter(filter_name, 29995, "lowpass", [400,425], comments= "adapted LFP filter for Banner20220105")

In [26]:
nwb_file_name


'Banner20220105_.nwb'

In [4]:
lfp_s_key
# filter = (FirFilterParameters() & {"filter_name": lfp_s_key["filter_name"]} & {"filter_sampling_rate": filter_sampling_rate}).fetch(as_dict=True)
# filter[0]
# filter = (
#             FirFilterParameters()
#             & {"filter_name": lfp_s_key["filter_name"]}
#             & {"filter_sampling_rate": sampling_rate}
#         ).fetch(as_dict=True)[0]

# from spyglass.common import Raw
# (Raw & {'nwb_file_name':nwb_file_name}).fetch1(
#             "sampling_rate", "interval_list_name"
#         )
# nwb_file_name
# Session() & {'nwb_file_name':nwb_file_name}
[x for x in Session().fetch('nwb_file_name') if 'Danvers' in x]
# nwb_file_name
skipped_sessions

['Banner20211228_.nwb',
 'Banner20211230_.nwb',
 'Banner20211231_.nwb',
 'Banner20220123_.nwb',
 'Banner20220124_.nwb',
 'Banner20220126_.nwb',
 'Banner20220215_.nwb',
 'Banner20220216_.nwb',
 'Banner20220223_.nwb',
 'Odins20220107_.nwb',
 'Odins20220110_.nwb',
 'Odins20220111_.nwb',
 'Odins20220113_.nwb',
 'Odins20220118_.nwb',
 'Odins20220119_.nwb',
 'Odins20220123_.nwb',
 'Odins20220124_.nwb',
 'Odins20220126_.nwb',
 'Odins20220216_.nwb']

# Define Tetrode Animals

In [61]:
tetrode_animals = [
    #Yoshi
    "Yoshi20220509_.nwb",
    "Yoshi20220510_.nwb",
    "Yoshi20220511_.nwb",
    "Yoshi20220513_.nwb",
    "Yoshi20220516_.nwb",
    "Yoshi20220517_.nwb",
    "Yoshi20220518_.nwb",
    "Yoshi20220519_.nwb",
    "Yoshi20220525_.nwb",
    #Totoro
    "Totoro20220524_.nwb",
    "Totoro20220525_.nwb",
    "Totoro20220526_.nwb",
    "Totoro20220527_.nwb",
    "Totoro20220530_.nwb",
    "Totoro20220531_.nwb",
    "Totoro20220602_.nwb",
    "Totoro20220603_.nwb",
    "Totoro20220606_.nwb",
    "Totoro20220607_.nwb",
    "Totoro20220610_.nwb",
    "Totoro20220613_.nwb",
    "Totoro20220614_.nwb",
    "Totoro20220615_.nwb",
    "Totoro20220616_.nwb",
    "Totoro20220617_.nwb",
    "Totoro20220621_.nwb",
    #Olive
    "Olive20220624_.nwb",
    "Olive20220627_.nwb",
    "Olive20220628_.nwb",
    "Olive20220629_.nwb",
    "Olive20220630_.nwb",
    "Olive20220701_.nwb",
    "Olive20220702_.nwb",
    "Olive20220705_.nwb",
    "Olive20220706_.nwb",
    "Olive20220707_.nwb",
    "Olive20220708_.nwb",
    "Olive20220711_.nwb",
    #Winnie
    "Winie20220712_.nwb",
    "Winie20220713_.nwb",
    "Winie20220714_.nwb",
    "Winie20220715_.nwb",
    "Winie20220716_.nwb",
    "Winie20220717_.nwb",
    "Winie20220718_.nwb",
    "Winie20220719_.nwb",
    "Winie20220720_.nwb",
    "Winie20220722_.nwb",
    #Wallie
    "Wallie20220911_.nwb",
    "Wallie20220912_.nwb",
    "Wallie20220913_.nwb",
    "Wallie20220914_.nwb",
    "Wallie20220915_.nwb",
    "Wallie20220916_.nwb",
    "Wallie20220922_.nwb",
]

# Trial Junkyard

In [ ]:
LFPElectrodeGroup().LFPElectrode() & {"nwb_file_name": nwb_file_name}
IntervalList& {"nwb_file_name": nwb_file_name,}
# interval_list[:4]
LFP()#Selection()& {'nwb_file_name': nwb_file_name}
# lfp_s_key
LFPBandSelection()
interval_list_name

{"lfp_id": lfp_id, "filter_name": filter_name , 'target_interval_list_name':interval_list_name}
LFP()
(LFP() & lfp_s_key).fetch_nwb()[0]["lfp"]
(LFPBandSelection() & lfp_b_key)
LFPBand().populate(LFPBandSelection() & lfp_b_key)

In [26]:
lfp_interval_list = LFPOutput().get_lfp_object({"lfp_id": lfp_id}).fetch1("interval_list_name")
lfp_valid_times = (
            IntervalList()
            & {
                "nwb_file_name": nwb_file_name,
                "interval_list_name": lfp_interval_list,
            }
        ).fetch1("valid_times")

valid_times = (
            IntervalList()
            & {
                "nwb_file_name": nwb_file_name,
                "interval_list_name": interval_list_name,
            }
        ).fetch1("valid_times")

epoch_interval = (IntervalList()& {
                "nwb_file_name": nwb_file_name,
                "interval_list_name": interval_list_name,
            })
min_length = (LFPBandSelection & lfp_b_key).fetch1("min_interval_len")
from functools import reduce

interval_list_intersect(
            valid_times, lfp_valid_times, min_length=min_length
        )

lfp_object = LFPOutput.get_lfp_object({"lfp_id": lfp_id})
timestamps = np.asarray(lfp_object.timestamps)

AttributeError: 'LFP' object has no attribute 'timestamps'

In [14]:
LFPOutput().get_lfp_object({'nwb_file_name':nwb_file_name,"lfp_id": lfp_id})
(LFPBandSelection & lfp_b_key).fetch1("min_interval_len")

1.0

In [12]:
print(lfp_valid_times,'\n',valid_times,'\n',epoch_interval.fetch1('valid_times'))
# LFPOutput().get_lfp_object({"lfp_id": lfp_id,})

[[1.64573092e+09 1.64573187e+09]
 [1.64573187e+09 1.64573213e+09]] 
 [[1.64573092e+09 1.64573213e+09]] 
 [[1.64573092e+09 1.64573213e+09]]


In [29]:
a = [1.64573187e+09, 1.64573187e+09]
print(a[1]-a[0])
lfp_valid_times[1][0]-lfp_valid_times[0][1]
lfp_object = (
            LFP() & {"nwb_file_name": nwb_file_name}
        ).fetch_nwb()[0]["lfp"]

[2023-06-30 16:26:14,399][WARNING]: Skipped checksum for file with hash: 8da8b253-bd8f-ca99-e332-91a506d6bac6, and path: /stelmo/nwb/analysis/Banner20220224/Banner20220224_VCWQJUGZXX.nwb
[2023-06-30 16:26:14,404][WARNING]: Skipped checksum for file with hash: 7bcb1c34-7c3a-54aa-4456-791f9c23b739, and path: /stelmo/nwb/analysis/Banner20220224/Banner20220224_ILSBCE5BER.nwb
[2023-06-30 16:26:14,408][WARNING]: Skipped checksum for file with hash: 05192d1d-1fe8-dff3-efea-1b2b3ecaaa3b, and path: /stelmo/nwb/analysis/Banner20220224/Banner20220224_L482ADMDTN.nwb
[2023-06-30 16:26:14,412][WARNING]: Skipped checksum for file with hash: a3c1af13-b53f-caf9-4ce1-3ee16276045d, and path: /stelmo/nwb/analysis/Banner20220224/Banner20220224_XFMG8I2JA3.nwb


0.0


In [30]:
LFP() & {"nwb_file_name": nwb_file_name}

nwb_file_name name of the NWB file,lfp_electrode_group_name the name of this group of electrodes,target_interval_list_name descriptive name of this interval list,filter_name descriptive name of this filter,filter_sampling_rate sampling rate for this filter,analysis_file_name name of the file,interval_list_name descriptive name of this interval list,lfp_object_id the NWB object ID for loading this object from the file,"lfp_sampling_rate the sampling rate, in HZ"
Banner20220224_.nwb,full_probe,pos 0 valid times,LFP 0-400 Hz,30000,Banner20220224_VCWQJUGZXX.nwb,full_probe_pos 0 valid times_valid times,0a0a6802-0724-4d4b-8985-d6732af52179,1000.0
Banner20220224_.nwb,full_probe,pos 1 valid times,LFP 0-400 Hz,30000,Banner20220224_ILSBCE5BER.nwb,full_probe_pos 1 valid times_valid times,b91f0624-43d8-4a3f-aeb5-c167fd98d3c3,1000.0
Banner20220224_.nwb,test_group,pos 3 valid times,LFP 0-400 Hz,30000,Banner20220224_L482ADMDTN.nwb,test_group_pos 3 valid times_valid times,9769ba64-6ecd-43bf-89cd-fccf32c9977b,1000.0
Banner20220224_.nwb,test_group,test interval,LFP 0-400 Hz,30000,Banner20220224_XFMG8I2JA3.nwb,test_group_test interval_valid times,ddfa2e79-b188-40bb-92cd-3698d35a204b,1000.0


In [23]:
from spyglass.common.common_interval import *